In [4]:
import pygad
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt 
import pandas as pd

In [5]:
easy_dataset_1 = pd.read_json("datasets/easy_dataset_1.json")
easy_dataset_1

,dataset_id,num_papers,num_reviewers,reviewer_capacity,min_reviews_per_paper,max_reviews_per_paper,preferences,friendships,authorship
0,Easy Dataset 1,5,5,3,3,5,"[3, 2, 1, 5, 3]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
1,Easy Dataset 1,5,5,3,3,5,"[5, 2, 4, 2, 2]","[0, 0, 1, 0, 0]","[0, 1, 0, 0, 0]"
2,Easy Dataset 1,5,5,3,3,5,"[4, 3, 1, 2, 4]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]"
3,Easy Dataset 1,5,5,3,3,5,"[4, 2, 2, 1, 1]","[0, 0, 0, 0, 0]","[1, 1, 0, 0, 1]"
4,Easy Dataset 1,5,5,3,3,5,"[4, 3, 4, 1, 5]","[0, 0, 0, 0, 0]","[0, 0, 1, 1, 0]"


In [30]:
N = easy_dataset_1['num_papers'][0]
K = easy_dataset_1['num_reviewers'][0]
R = easy_dataset_1['reviewer_capacity'][0]
RPP = np.array([easy_dataset_1['min_reviews_per_paper'][0],easy_dataset_1['max_reviews_per_paper'][0]])
Preference = np.array([sublist for sublist in easy_dataset_1['preferences']])
print(Preference)
Frendship = np.array([sublist for sublist in easy_dataset_1['friendships']],dtype=bool)
print(Frendship)
Autorship = np.array([sublist for sublist in easy_dataset_1['authorship']],dtype=bool)
print(Autorship)

[[3 2 1 5 3]
 [5 2 4 2 2]
 [4 3 1 2 4]
 [4 2 2 1 1]
 [4 3 4 1 5]]
[[False False False False False]
 [False False  True False False]
 [False  True False False False]
 [False False False False False]
 [False False False False False]]
[[False False False False False]
 [False  True False False False]
 [False False False False False]
 [ True  True False False  True]
 [False False  True  True False]]


In [86]:
gene_space = [True,False]
example_solution = np.zeros(Preference.shape,dtype=bool)
example_solution[0:4][1:4] = True
example_solution

array([[False, False, False, False, False],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [False, False, False, False, False]])

In [83]:
def GA_fitness_function(ga_instance, solution, solution_idx):
    return fitness_function(solution)
def fitness_function(solution):
    Pref = Preference * solution
    Pref = sum(sum(Pref))
    Frend = (Frendship * solution).astype(int) 
    Frend = sum(sum(Frend))
    Auth = (Autorship * solution).astype(int) 
    Auth = sum(sum(Auth))
    revievs_per_paper = (sum(solution) >= RPP[0] )*(sum(solution) <= RPP[1])
    revievs_per_paper = sum(revievs_per_paper - 1)
    print(solution.sum(axis=0))
    return Pref -10* Frend -20*Auth + 10*revievs_per_paper

In [87]:
print(fitness_function(example_solution))

[3 3 3 3 3]
-61
